<a href="https://colab.research.google.com/github/nagusubra/Solar_panel_dust_detection/blob/main/Solar_panel_dust_detection_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization
!pip install openpyxl
!pip install xlsxwriter

import xlsxwriter
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 20.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 15.6 MB/s eta 0:00:00


In [2]:

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import cross_val_score

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

import os
import zipfile

In [3]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation function

In [4]:
# Evaluate Model Size
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)


def evaluate_model(model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df


def evaluate_model_without_saving_stats(model, model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  # model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df




# research paper evaluation function
def evaluate_research_paper_with_model_path(model_path):
  # model_path = "/content/models/solnet.hdf5"
  solnet = load_model(model_path, compile=False)
  history = solnet.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

def evaluate_research_paper_with_model(model):
  history = model.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

# Load base model and Evaluate


In [5]:
final_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/solnetOriginal_final_base_model.hdf5'
final_model = tf.keras.models.load_model(final_model_path)

In [6]:
batch_size = 32
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)


Found 1440 files belonging to 2 classes.
Using 1152 files for training.
Found 1440 files belonging to 2 classes.
Using 288 files for validation.


In [7]:
test_acc, model_size, executionTime, evulation_df = evaluate_model_without_saving_stats(final_model, final_model_path, "#0", val_dataset)
print(test_acc)

9/9 [==============================] - 24s 33ms/step

Model Accuracy: 72.56944179534912 %
Model Size: 659509171.00 bytes
Inference Time is:  4.570444742838542 s
0.7256944179534912


In [8]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,72.56944179534912 %
3,Loss,261.79609298706055 %
4,Model Size,659509171 bytes
5,Inference Time,4.570444742838542 sec


# Compression

In [9]:
def iterative_pruning(model, initial_sparsity, final_sparsity, begin_step, end_step, train_images, train_labels, epochs):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Define model for pruning.
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
        final_sparsity=final_sparsity, begin_step=begin_step, end_step=end_step, frequency=100)
  }

  pruned_model = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  pruned_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])


  callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
  ]

  pruned_model.fit(train_images, train_labels, epochs=epochs, validation_split=0.1,
                    callbacks=callbacks)

  # Strip pruning wrappers
  stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

  return pruned_model, stripped_pruned_model

In [15]:
type(tr_dataset)

tensorflow.python.data.ops.batch_op._BatchDataset

In [52]:
import tensorflow_datasets as tfds

x_train = None
x_train_labels = None

for image, label in tfds.as_numpy(tr_dataset):
  # print(type(image), type(label), label, len(label))
  x_train = image
  x_train_labels = label
  
  print(type(x_train))
  print(len(x_train))
  print(x_train.ndim)
  print(x_train.shape)

  
  print('------')

<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)
------
<class 'numpy.ndarray'>
32
4

In [29]:
x_test = None

for image, label in tfds.as_numpy(val_dataset):
  print(type(image), type(label), label, len(label))
  x_test = image
  x_test_labels = label
  print('------')

<class 'numpy.ndarray'> <class 'numpy.ndarray'> [[1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]] 32
------
<class 'numpy.ndarray'> <class 'numpy.ndarray'> [[1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]] 32
------
<class 'numpy.ndarray'> <class 'numpy.ndarray'> [[1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]] 32
------
<class 'numpy.ndarray'> <class 'numpy.ndarray'> [[1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]] 32
---

In [38]:
print(type(x_train))
print(len(x_train))
print(x_train.ndim)
print(x_train.shape)


<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)


In [50]:
x_train_images = np.empty(0)
x_train_labels = np.empty(0)

for image, label in tfds.as_numpy(tr_dataset):
  # print(type(image), type(label), label, len(label))
  x_train_images = np.append(x_train_images, image)
  x_train_labels = np.append(x_train_labels, label)
  # print('------')


In [44]:

print(type(x_train_images))
print(len(x_train_images))
print(x_train_images.ndim)
print(x_train_images.shape)

<class 'numpy.ndarray'>
178084224
1
(178084224,)


In [39]:
print(type(x_train))
print(len(x_train))
print(x_train.ndim)
print(x_train.shape)


<class 'numpy.ndarray'>
32
4
(32, 227, 227, 3)


In [20]:
x_train

array([[[[ 89.32104   , 103.3761    , 126.06112   ],
         [ 98.20719   , 105.15212   , 133.98691   ],
         [ 95.99023   , 102.99023   , 131.99023   ],
         ...,
         [102.10312   ,  95.10312   ,  76.10312   ],
         [133.23564   , 124.29034   , 115.18096   ],
         [126.924995  , 119.924995  , 100.924995  ]],

        [[ 95.        , 102.31498   , 127.37004   ],
         [100.        , 107.        , 135.8348    ],
         [ 95.        , 102.        , 131.        ],
         ...,
         [186.81335   , 179.81335   , 160.81335   ],
         [157.67924   , 150.67924   , 131.67924   ],
         [114.85003   , 107.85003   ,  88.85003   ]],

        [[100.        , 107.31498   , 132.37004   ],
         [ 98.53084   , 105.53084   , 134.36563   ],
         [101.14978   , 108.14978   , 137.14978   ],
         ...,
         [115.76187   , 108.76187   ,  89.76187   ],
         [145.74023   , 138.74023   , 119.74023   ],
         [114.64123   , 107.64123   ,  88.64123   ]],

In [51]:
# frquency = 100
# target sparsity = 50%
# sparisty for each pruning tep = 50%/100 = 0.05%
# Total Number of Training Samples = 1440
# batch size = 32
# epochs = 3

# possible start and end for the iterative pruning schedule
# Number of learning steps = (1440/32)*3 = 135 (15, 120)
# Number of learning steps = (1440/32)*4 = 180 (, )
# Number of learning steps = (1440/32)*5 = 225 (, )
# Number of learning steps = (1440/32)*6 = 270 (, )

# iterative_pruning(
                  #     model, 
                  #     initial_sparsity = 0, 
                  #     final_sparsity   = 0.9, 
                  #     begin_step       = 0, 600  (~10% of 5625) # pruning early on the pruning schedule to get optimal model size
                  #     end_step         = ?, 4500 (~80% of 5625) # pruning only till 80% so that model can still have enough leanring steps to learn and build networks for the data
                  #     train_images, 
                  #     train_labels, 
                  #     epochs           = 3, no change ? since the Number of Steps per Epoch = (Total Number of Training Samples = ?) / (Batch Size = ?) 
                  # )

pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning(final_model, 0, 0.5, 15, 120, x_train_images, x_train_labels, 3)

ValueError: ignored

In [46]:
stripped_pruned_model_saprsity_50.save('stripped_pruned_model_saprsity_50.h5')

In [47]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )


print_model_weights_sparsity(pruned_model_saprsity_50)

conv2d/kernel:0: 0.00% sparsity  (0/34848)
conv2d_1/kernel:0: 0.00% sparsity  (0/614400)
conv2d_2/kernel:0: 0.00% sparsity  (0/884736)
conv2d_3/kernel:0: 0.00% sparsity  (0/1327104)
conv2d_4/kernel:0: 0.00% sparsity  (0/884736)
dense/kernel:0: 0.00% sparsity  (0/37748736)
dense_1/kernel:0: 0.00% sparsity  (0/16777216)
dense_2/kernel:0: 0.00% sparsity  (0/4096)


In [48]:
test_loss_pruned_50, test_acc_pruned_50 = pruned_model_saprsity_50.evaluate(x_test,  x_test_labels, verbose=0)

In [49]:
startTime = time.time()
prediction = pruned_model_saprsity_50.predict(x_test)
executionTimePruned50 = (time.time() - startTime)/len(x_test)

1/1 [==============================] - 0s 40ms/step


In [33]:
pruned_model_size = get_gzipped_model_size('stripped_pruned_model_saprsity_50.h5')

In [34]:
print('\nPruned Model Accuracy:', test_acc_pruned_50*100, '%')
print("Pruned Model Size: %.2f bytes" % (pruned_model_size))
print("Pruned Inference Time is", executionTimePruned50, "s")


Pruned Model Accuracy: 43.75 %
Pruned Model Size: 244122.00 bytes
Pruned Inference Time is 0.01014922559261322 s


In [35]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,72.56944179534912 %
3,Loss,261.79609298706055 %
4,Model Size,659509171 bytes
5,Inference Time,4.570444742838542 sec
